In [ ]:
##라이더 할당 수정##
# def assign_riders_with_weighted_probability(riders, effectiveness_indicator):
#     # 가중치에 따라 라이더 타입을 선택
#     total_effectiveness = sum([indicator[1] for indicator in effectiveness_indicator])
#     weights = [indicator[1] / total_effectiveness for indicator in effectiveness_indicator]

#     # 라이더 타입 리스트와 그에 해당하는 가중치 리스트를 생성
#     rider_types = [indicator[0] for indicator in effectiveness_indicator]

#     while rider_types:
#         selected_type = random.choices(rider_types, weights=weights, k=1)[0].upper()
        
#         # 해당 타입의 가용 가능한 라이더 찾기
#         selected_rider = next(
#             (rider for rider in riders if rider.type.upper() == selected_type and rider.available_number > 0),
#             None
#         )
        
#         if selected_rider:
#             return selected_rider
#         else:
#             # 선택된 타입의 라이더가 가용 가능하지 않다면, 해당 타입을 리스트에서 제거
#             index = rider_types.index(selected_type)  # 원래 타입을 제거할 때 사용
#             rider_types.pop(index)
#             weights.pop(index)
#             # 가중치를 다시 계산 (남은 가중치의 합이 1이 되도록)
#             if weights:
#                 total_weights = sum(weights)
#                 weights = [w / total_weights for w in weights]

#     # 만약 모든 타입의 라이더가 가용하지 않다면 None 반환
#     return None

# def single_run_algorithm(K, all_orders, all_riders, dist_mat, timelimit=60):
#     start_time = time.time()

#     for r in all_riders:
#         r.T = np.round(dist_mat / r.speed + r.service_time)

#     # 효율성 지표 (예시로 임의의 값 사용)
#     #effectiveness_indicator = [["bike", 100], ["car", 500], ["walk", 300]]
#     effectiveness_indicator = calculate_efficiencies(K, all_riders, all_orders, dist_mat)
#     #effectiveness_dict = {rider.type: effectiveness for rider, effectiveness in zip(all_riders, effectiveness_indicator)}

#     # 주문들을 ready_time 기준으로 정렬
#     #sorted_orders = sorted(all_orders, key=lambda order: order.ready_time)
#     sorted_orders = random.sample(all_orders, len(all_orders))

#     # 모든 라이더를 합쳐서 처리
#     all_riders_list = all_riders
#     print(all_riders_list)

#     all_bundles = []

#     while sorted_orders and all_riders_list:
#         rider = assign_riders_with_weighted_probability(all_riders_list, effectiveness_indicator)
#         if rider.available_number > 0:
#             bundles, sorted_orders = assign_orders_to_rider(rider, sorted_orders, dist_mat, K, all_orders)
#             for bundle in bundles:
#                 all_bundles.append(bundle)


#     best_obj = sum((bundle.cost for bundle in all_bundles)) / K
#     print(f'Initial best obj = {best_obj}')


#     solution = [
#         [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
#         for bundle in all_bundles
#     ]

#     return solution, best_obj

In [ ]:
##번들에 하나의 주문만 있는 경우, 더 탐색##
# def assign_orders_to_rider(rider, orders, dist_mat, K, all_orders):
#     bundles = []
#     remaining_orders = orders[:]
    
#     while remaining_orders and rider.available_number > 0:
#         current_order = remaining_orders.pop(0)
#         current_bundle = [current_order]
#         shop_seq = [current_order.id]
#         delivery_seq = sorted(shop_seq, key=lambda order_id: all_orders[order_id].deadline)
        
#         is_feasible = test_route_feasibility(all_orders, rider, shop_seq, delivery_seq)
#         if is_feasible != 0:
#             remaining_orders.insert(0, current_order)
#             return bundles, remaining_orders
        
#         current_volume = current_order.volume
#         current_time = current_order.ready_time

#         while True:
#             nearest_orders = find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, 30)
#             added = False

#             if len(current_bundle) >= 4:
#                 break
            
#             for next_order in nearest_orders:
#                 if current_volume + next_order.volume > rider.capa:
#                     continue
                
#                 current_bundle_ids = [o.id for o in current_bundle]
#                 next_bundle_ids = [next_order.id]

#                 combined_ids = current_bundle_ids + next_bundle_ids
#                 pickup_permutations = itertools.permutations(combined_ids)

#                 valid_combinations = []
                
#                 for perm_shop_seq in pickup_permutations:
#                     delivery_permutations = itertools.permutations(perm_shop_seq)
#                     for perm_dlv_seq in delivery_permutations:
#                         new_bundle = Bundle(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq), current_volume + next_order.volume, 0)
#                         new_bundle.total_dist = get_total_distance(K, dist_mat, list(perm_shop_seq), list(perm_dlv_seq))
#                         new_bundle.update_cost()

#                         is_feasible = test_route_feasibility(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq))
#                         if is_feasible == 0:
#                             valid_combinations.append((list(perm_shop_seq), list(perm_dlv_seq)))

#                 if valid_combinations:
#                     best_combination = min(valid_combinations, key=lambda x: get_total_distance(K, dist_mat, x[0], x[1]))
#                     best_shop_seq, best_dlv_seq = best_combination

#                     current_bundle.append(next_order)
#                     current_volume += next_order.volume
#                     current_time += rider.T[current_bundle[-2].id, next_order.id]
#                     remaining_orders.remove(next_order)
#                     added = True

#                     # 선택된 best_shop_seq와 best_dlv_seq로 번들 갱신
#                     shop_seq = best_shop_seq
#                     delivery_seq = best_dlv_seq
#                     break

#             if not added:
#                 # nearest_orders에서 찾지 못한 경우, remaining_orders의 나머지 주문들 중에서 가까운 30개 찾기
#                 other_orders = [order for order in remaining_orders if order not in nearest_orders]
                
#                 # other_orders에서 current_bundle과 가장 가까운 30개 주문 선택
#                 if other_orders:
#                     other_orders = find_nearest_orders(current_bundle, other_orders, dist_mat, K, 30)

#                 for next_order in other_orders:
#                     if current_volume + next_order.volume > rider.capa:
#                         continue

#                     current_bundle_ids = [o.id for o in current_bundle]
#                     next_bundle_ids = [next_order.id]

#                     combined_ids = current_bundle_ids + next_bundle_ids
#                     pickup_permutations = itertools.permutations(combined_ids)

#                     valid_combinations = []

#                     for perm_shop_seq in pickup_permutations:
#                         delivery_permutations = itertools.permutations(perm_shop_seq)
#                         for perm_dlv_seq in delivery_permutations:
#                             new_bundle = Bundle(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq), current_volume + next_order.volume, 0)
#                             new_bundle.total_dist = get_total_distance(K, dist_mat, list(perm_shop_seq), list(perm_dlv_seq))
#                             new_bundle.update_cost()

#                             is_feasible = test_route_feasibility(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq))
#                             if is_feasible == 0:
#                                 valid_combinations.append((list(perm_shop_seq), list(perm_dlv_seq)))

#                     if valid_combinations:
#                         best_combination = min(valid_combinations, key=lambda x: get_total_distance(K, dist_mat, x[0], x[1]))
#                         best_shop_seq, best_dlv_seq = best_combination

#                         current_bundle.append(next_order)
#                         current_volume += next_order.volume
#                         current_time += rider.T[current_bundle[-2].id, next_order.id]
#                         remaining_orders.remove(next_order)
#                         added = True

#                         # 선택된 best_shop_seq와 best_dlv_seq로 번들 갱신
#                         shop_seq = best_shop_seq
#                         delivery_seq = best_dlv_seq
#                         break

#             if not added:
#                 break

#         final_bundle = Bundle(all_orders, rider, shop_seq, delivery_seq, current_volume, get_total_distance(K, dist_mat, shop_seq, delivery_seq))
#         bundles.append(final_bundle)
#         rider.available_number -= 1

#     return bundles, remaining_orders

In [1]:
#다른 타입의 라이더도 다 확인#
def assign_orders_to_rider(rider, orders, dist_mat, K, all_orders, all_riders):
    bundles = []
    remaining_orders = orders[:]
    
    while remaining_orders and rider.available_number > 0:
        current_order = remaining_orders.pop(0)
        current_bundle = [current_order]
        shop_seq = [current_order.id]
        delivery_seq = sorted(shop_seq, key=lambda order_id: all_orders[order_id].deadline)
        
        is_feasible = test_route_feasibility(all_orders, rider, shop_seq, delivery_seq)
        if is_feasible != 0:
            remaining_orders.insert(0, current_order)
            return bundles, remaining_orders
        
        current_volume = current_order.volume
        current_time = current_order.ready_time

        while True:
            nearest_orders = find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, 50)
            added = False

            if len(current_bundle) >= 4:
                break
            
            for next_order in nearest_orders:
                if current_volume + next_order.volume > rider.capa:
                    continue
                
                current_bundle_ids = [o.id for o in current_bundle]
                next_bundle_ids = [next_order.id]

                combined_ids = current_bundle_ids + next_bundle_ids
                pickup_permutations = itertools.permutations(combined_ids)

                valid_combinations = []
                
                for perm_shop_seq in pickup_permutations:
                    delivery_permutations = itertools.permutations(perm_shop_seq)
                    for perm_dlv_seq in delivery_permutations:
                        new_bundle = Bundle(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq), current_volume + next_order.volume, 0)
                        new_bundle.total_dist = get_total_distance(K, dist_mat, list(perm_shop_seq), list(perm_dlv_seq))
                        new_bundle.update_cost()

                        is_feasible = test_route_feasibility(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq))
                        if is_feasible == 0:
                            valid_combinations.append((list(perm_shop_seq), list(perm_dlv_seq)))

                if valid_combinations:
                    best_combination = min(valid_combinations, key=lambda x: get_total_distance(K, dist_mat, x[0], x[1]))
                    best_shop_seq, best_dlv_seq = best_combination

                    current_bundle.append(next_order)
                    current_volume += next_order.volume
                    current_time += rider.T[current_bundle[-2].id, next_order.id]
                    remaining_orders.remove(next_order)
                    added = True

                    # 선택된 best_shop_seq와 best_dlv_seq로 번들 갱신
                    shop_seq = best_shop_seq
                    delivery_seq = best_dlv_seq
                    break

            if not added:
                break

        # 현재 라이더로 final_bundle을 생성하기 전에 다른 라이더도 확인
        final_bundle = Bundle(all_orders, rider, shop_seq, delivery_seq, current_volume, get_total_distance(K, dist_mat, shop_seq, delivery_seq))
        min_cost = final_bundle.total_dist * rider.var_cost + rider.fixed_cost
        best_rider = rider

        # 다른 타입의 라이더들에 대해서도 비용 계산 및 비교
        for r in all_riders:
            if r.type != rider.type and r.available_number > 0:
                test_bundle = Bundle(all_orders, r, shop_seq, delivery_seq, current_volume, get_total_distance(K, dist_mat, shop_seq, delivery_seq))
                if test_route_feasibility(all_orders, r, shop_seq, delivery_seq) == 0:
                    test_cost = test_bundle.total_dist * r.var_cost + r.fixed_cost
                    if test_cost < min_cost:
                        min_cost = test_cost
                        best_rider = r
                        final_bundle = test_bundle

        bundles.append(final_bundle)
        best_rider.available_number -= 1

    return bundles, remaining_orders

In [2]:
#bike 라이더 우선할당#
def single_run_algorithm(K, all_orders, all_riders, dist_mat, timelimit=60):
    start_time = time.time()

    for r in all_riders:
        r.T = np.round(dist_mat / r.speed + r.service_time)

    # 효율성 지표 계산
    effectiveness_indicator = calculate_efficiencies(K, all_riders, all_orders, dist_mat)

    # 주문들을 무작위로 섞어 정렬
    sorted_orders = random.sample(all_orders, len(all_orders))

    # 모든 라이더를 합쳐서 처리
    all_riders_list = all_riders

    all_bundles = []

    # while sorted_orders and all_riders_list:
    #     rider = assign_riders_with_weighted_probability(all_riders_list, effectiveness_indicator)
    #     if rider.available_number > 0:
    #         bundles, sorted_orders = assign_orders_to_rider(rider, sorted_orders, dist_mat, K, all_orders, all_riders)
    #         for bundle in bundles:
    #             all_bundles.append(bundle)

    bike_riders = [r for r in all_riders if r.type == 'BIKE']
    car_riders = [r for r in all_riders if r.type == 'CAR']
    walk_riders = [r for r in all_riders if r.type == 'WALK']

    all_bundles = []

    # BIKE 라이더에게 우선 할당
    for bike_rider in bike_riders:
        if not sorted_orders:
            break
        if bike_rider.available_number > 0:
            bundles, sorted_orders = assign_orders_to_rider(bike_rider, sorted_orders, dist_mat, K, all_orders, all_riders)
            for bundle in bundles:
                #if test_route_feasibility(all_orders, bike_rider, bundle.shop_seq, bundle.dlv_seq) == 0:
                    all_bundles.append(bundle)

    # 남은 주문들을 CAR 또는 WALK 라이더에게 할당
    remaining_riders = car_riders + walk_riders
    random.shuffle(remaining_riders)
    while sorted_orders and remaining_riders:
        rider = remaining_riders.pop(0)
        if rider.available_number > 0:
            bundles, sorted_orders = assign_orders_to_rider(rider, sorted_orders, dist_mat, K, all_orders, all_riders)
            for bundle in bundles:
                #if test_route_feasibility(all_orders, rider, bundle.shop_seq, bundle.dlv_seq) == 0:
                    all_bundles.append(bundle)

    # 최적의 비용 계산
    best_obj = sum((bundle.cost for bundle in all_bundles)) / K
    print(f'Initial best obj = {best_obj}')

    best_obj = sum((bundle.cost for bundle in all_bundles)) / K
    print(f'Optimized obj after merging single-order bundles = {best_obj}')

    solution = [
        [bundle.rider.type, bundle.shop_seq, bundle.dlv_seq]
        for bundle in all_bundles
    ]

    return solution, best_obj

In [3]:
#병렬 계산 orginal#
# def algorithm(K, all_orders, all_riders, dist_mat, timelimit=60, num_processes=61):
#     with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
#         futures = [executor.submit(single_run_algorithm, K, all_orders, all_riders, dist_mat, timelimit) for _ in range(num_processes)]
#         results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
#     # 각 프로세스의 목적함수 출력
#     for i, result in enumerate(results):
#         solution, obj_value = result
#         print(f'Objective value from process {i+1}: {obj_value}')
    
#     # 최적의 solution 선택
#     best_solution = min(results, key=lambda x: x[1])
#     return best_solution[0]

In [4]:
#nearest_order를 deadline 차이가 큰 순으로 정렬)
def assign_orders_to_rider(rider, orders, dist_mat, K, all_orders, all_riders):
    bundles = []
    remaining_orders = orders[:]
    
    while remaining_orders and rider.available_number > 0:
        current_order = remaining_orders.pop(0)
        current_bundle = [current_order]
        shop_seq = [current_order.id]
        delivery_seq = sorted(shop_seq, key=lambda order_id: all_orders[order_id].deadline)
        
        is_feasible = test_route_feasibility(all_orders, rider, shop_seq, delivery_seq)
        if is_feasible != 0:
            remaining_orders.insert(0, current_order)
            return bundles, remaining_orders
        
        current_volume = current_order.volume
        current_time = current_order.ready_time

        while True:
            nearest_orders = find_nearest_orders(current_bundle, remaining_orders, dist_mat, K, 50)

            nearest_orders.sort(
                key=lambda order: abs(current_order.deadline - order.deadline),
                reverse=True
            ) 

            added = False

            if len(current_bundle) >= 4:
                break
            
            for next_order in nearest_orders:
                if current_volume + next_order.volume > rider.capa:
                    continue
                
                current_bundle_ids = [o.id for o in current_bundle]
                next_bundle_ids = [next_order.id]

                combined_ids = current_bundle_ids + next_bundle_ids
                pickup_permutations = itertools.permutations(combined_ids)

                valid_combinations = []
                
                for perm_shop_seq in pickup_permutations:
                    delivery_permutations = itertools.permutations(perm_shop_seq)
                    for perm_dlv_seq in delivery_permutations:
                        new_bundle = Bundle(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq), current_volume + next_order.volume, 0)
                        new_bundle.total_dist = get_total_distance(K, dist_mat, list(perm_shop_seq), list(perm_dlv_seq))
                        new_bundle.update_cost()

                        is_feasible = test_route_feasibility(all_orders, rider, list(perm_shop_seq), list(perm_dlv_seq))
                        if is_feasible == 0:
                            valid_combinations.append((list(perm_shop_seq), list(perm_dlv_seq)))

                if valid_combinations:
                    best_combination = min(valid_combinations, key=lambda x: get_total_distance(K, dist_mat, x[0], x[1]))
                    best_shop_seq, best_dlv_seq = best_combination

                    current_bundle.append(next_order)
                    current_volume += next_order.volume
                    current_time += rider.T[current_bundle[-2].id, next_order.id]
                    remaining_orders.remove(next_order)
                    added = True

                    # 선택된 best_shop_seq와 best_dlv_seq로 번들 갱신
                    shop_seq = best_shop_seq
                    delivery_seq = best_dlv_seq
                    break

            if not added:
                break

        final_bundle = Bundle(all_orders, rider, shop_seq, delivery_seq, current_volume, get_total_distance(K, dist_mat, shop_seq, delivery_seq))
        bundles.append(final_bundle)
        rider.available_number -= 1

    return bundles, remaining_orders
